## Sucesses and Failures of Hartree-Fock


In this example, we will quantify the different sources for error of the Hartree Fock method on the example of the H$_2$ molecule.

In [1]:
from ase.build import molecule
from ase.atoms import Atoms
import numpy as np
import matplotlib.pyplot as plt
import psi4
from psi4_helper import geom_ase_to_psi4

diatomic = 'C2'

In [ ]:
print('.', end='')
atoms = Atoms(diatomic)
atoms.positions[1,2] = 1.5
geom_input = geom_ase_to_psi4(atoms, charge=0, multiplicity=1)
  #initiate Psi4 molecule object
h2 = psi4.geometry(geom_input)

  # Set computation options
psi4.set_options({'basis': '3-21G',
                  'reference': 'rhf',
                  'scf_type':'pk',
                  #'frozen_docc': [0],
                  #'active':[4],
                  #'frozen_uocc': [0],
                  'e_convergence': 1e-8})
E,wfn = psi4.energy('scf',return_wfn=True)
psi4.molden(wfn,'c2.molden')

In [4]:
psi4.core.clean()
psi4.core.set_output_file('output.dat', False)
psi4.set_memory('6000 MB')

distances = [0.4,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.4,1.8,2.2,3.0,3.5,4.5,6,10.]

E_HF = []
E_exact = []

for i in distances:
  print('. ', end='')
  atoms = Atoms(diatomic)
  atoms.positions[1,2] = i
  geom_input = geom_ase_to_psi4(atoms, charge=0, multiplicity=1)
  #initiate Psi4 molecule object
  h2 = psi4.geometry(geom_input)

  # Set computation options
  psi4.set_options({'basis': '3-21G',
                  'reference': 'rhf',
                  'scf_type':'direct',
                  'frozen_docc': [2],
                  'active':[14],
                  'frozen_uocc': [2],
                  'ci_maxiter': 100,
                  'e_convergence': 1e-8})

  E_HF.append(psi4.energy('scf',return_wfn=False))
  #E_exact.append(psi4.energy('fci',return_wfn=False))
  E_exact.append(0)
  print (i, E_HF[-1], E_exact[-1])

print('calculations finished!')
#for x, i,j in zip(distances,E_HF, E_exact):
#    print(x, i,j)

. 0.4 -66.84414714958633 0
. 0.6 -72.16352508075667 0
. 0.7 -73.3640305088472 0
. 0.8 -74.1145656053884 0
. 0.9 -74.36058272141038 0
. 1.0 -74.68254386670876 0
. 1.1 -74.90594474340683 0
. 1.2 -74.94481910775252 0
. 1.4 -74.98506898837397 0
. 1.8 -74.90922754407586 0
. 2.2 -74.80553148039115 0
. 3.0 -74.63762210102273 0
. 3.5 -74.78140482776837 0
. 4.5 -74.49490079173465 0
. 6 -74.45590823843044 0
. 10.0 -74.78273158707306 0
calculations finished!


In [ ]:
%matplotlib notebook
plt.plot(distances, E_HF,'b',label='HF')
plt.plot(distances, E_exact,'r',label='exact')
plt.legend()
plt.xlabel('H-H distance in Angstrom $\AA$')
plt.ylabel('energy in Hartree [$E_H$]')

The exact solution includes all possible Slater determinants and correctly describes bond breaking. This is, however, in almost all cases impossible to calculate.

We can find a solution to the correlation problem in two steps:
* first we account for static correlation in the bond breaking limit. This will treat all H-H distances equally
* Then we can include dynamic correlation

In [5]:
E_CASSCF = []

psi4.core.set_output_file('output.dat', True)
distances = [0.4,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.4,1.8,2.2,3.0,3.5,4.5,6,10.]

for i in distances:
  psi4.core.clean()
  print('. ', end='')
  atoms = Atoms(diatomic)
  atoms.positions[1,2] = i
  geom_input = geom_ase_to_psi4(atoms, charge=0, multiplicity=1)
  #initiate Psi4 molecule object
  h2 = psi4.geometry(geom_input)

  # Set computation options
  psi4.set_options({'basis': '3-21G',
                  'reference': 'rhf',
                  'scf_type': 'pk',
                  'mcscf_algorithm':'ah',
                  'qc_module': 'detci',
                  'ci_maxiter': 100,    
                    'mcscf_maxiter': 300,
                  'e_convergence': 1.e-8,
                  'num_roots':1,
                   })

  #cisd_energy, cisd_wfn = psi4.energy("cisd", return_wfn=True)
  #4 orbitals [sigma, sigma*, and two more), The first 2 of them should be active
  #That means that CASSCF should build the possible determinants from those two
  psi4.set_options({
      'restricted_docc': [3],
      'active': [7],
      'restricted_uocc':[8],
                   })
    
  E = psi4.energy('casscf')#,ref_wfn=cisd_wfn)
  print(i, E)
  E_CASSCF.append(E) 

print('calculations finished!')
for x, i in zip(distances,E_CASSCF):
    print(x, i)


. 0.4 -66.87320040165278
. 0.6 -72.2669107049573
. 0.7 -73.52363948583401
. 0.8 -74.3080355683953
. 0.9 -74.76345382177082
. 1.0 -75.00723797334713
. 1.1 -75.12576799662014
. 1.2 -75.17115169050629
. 1.4 -75.15747304399395
. 1.8 -75.04873846714926
. 2.2 -75.0007399833979
. 3.0 -74.97229116739274
. 3.5 -74.96835668920913
. 4.5 -74.96610368675915
. 6 -74.96591375733942
. 

RuntimeError: 
Fatal Error: Iterations did not converge.
Error occurred in file: /scratch/psilocaluser/conda-builds/psi4_1495014881710/work/psi4/src/psi4/libparallel/process.cc on line: 194
The most recent 5 function calls were:

psi::PsiException::PsiException(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, char const*, int)
psi::die_if_not_converged()
psi::scf::HF::finalize_E()
psi::scf::HF::compute_energy()


In [ ]:
%matplotlib notebook
plt.plot(distances, E_HF,'b',label='HF')
plt.plot(distances, E_CASSCF, 'g', label='CASSCF')
plt.plot(distances, E_exact,'r',label='exact')
plt.legend()
plt.xlabel('H-H distance in Angstrom $\AA$')
plt.ylabel('energy in Hartree [$E_H$]')